#  Importing Important Packages

In [ ]:
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import re
import spacy
from nltk.corpus import sentiwordnet as swn
from IPython.display import clear_output
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import plotly
plotly.offline.init_notebook_mode (connected = True)
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk import ngrams
# The following code creates a word-document matrix.
from sklearn.feature_extraction.text import CountVectorizer
# Modeling packages
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

# Reading Data

In [ ]:
data=pd.read_csv('../input/trip-advisor-hotel-reviews/tripadvisor_hotel_reviews.csv')

In [ ]:
data.shape

In [ ]:
data.head(5)

# Making two copies of Reviews to edit

In [ ]:
#Edits After Removing Stopwords
Edited_Review = data['Review'].copy()

# Having a look at 1st ten reviews in the data

In [ ]:
data.head(10)

# Preprocessing Function

In [ ]:
# Function to preprocess the Reviews data
def preprocess_Reviews_data(data,name):
    # Proprocessing the data
    data[name]=data[name].str.lower()
    # Code to remove the Hashtags from the text
    data[name]=data[name].apply(lambda x:re.sub(r'\B#\S+','',x))
    # Code to remove the links from the text
    data[name]=data[name].apply(lambda x:re.sub(r"http\S+", "", x))
    # Code to remove the Special characters from the text 
    data[name]=data[name].apply(lambda x:' '.join(re.findall(r'\w+', x)))
    # Code to substitute the multiple spaces with single spaces
    data[name]=data[name].apply(lambda x:re.sub(r'\s+', ' ', x, flags=re.I))
    # Code to remove all the single characters in the text
    data[name]=data[name].apply(lambda x:re.sub(r'\s+[a-zA-Z]\s+', '', x))
    # Remove the twitter handlers
    data[name]=data[name].apply(lambda x:re.sub('@[^\s]+','',x))

# Function to tokenize and remove the stopwords    
def rem_stopwords_tokenize(data,name):
      
    def getting(sen):
        example_sent = sen
        
        filtered_sentence = [] 

        stop_words = set(stopwords.words('english')) 

        word_tokens = word_tokenize(example_sent) 
        
        filtered_sentence = [w for w in word_tokens if not w in stop_words] 
        
        return filtered_sentence
    # Using "getting(sen)" function to append edited sentence to data
    x=[]
    for i in data[name].values:
        x.append(getting(i))
    data[name]=x


# Lemmatization Function

In [ ]:
lemmatizer = WordNetLemmatizer()
def Lemmatization(data,name):
    def getting2(sen):
        
        example = sen
        output_sentence =[]
        word_tokens2 = word_tokenize(example)
        lemmatized_output = [lemmatizer.lemmatize(w) for w in word_tokens2]
        
        # Remove characters which have length less than 2  
        without_single_chr = [word for word in lemmatized_output if len(word) > 2]
        # Remove numbers
        cleaned_data_title = [word for word in without_single_chr if not word.isnumeric()]
        
        return cleaned_data_title
    # Using "getting2(sen)" function to append edited sentence to data
    x=[]
    for i in data[name].values:
        x.append(getting2(i))
    data[name]=x

# Converting all the texts back to sentences

In [ ]:
def make_sentences(data,name):
    data[name]=data[name].apply(lambda x:' '.join([i+' ' for i in x]))
    # Removing double spaces if created
    data[name]=data[name].apply(lambda x:re.sub(r'\s+', ' ', x, flags=re.I))

In [ ]:
# Using the Lemmatization function to lemmatize the Reviews data
data['Review_without_stopwords'] = Edited_Review

# Using the preprocessing function to preprocess the hotel data
preprocess_Reviews_data(data,'Review_without_stopwords')
# Using tokenizer and removing the stopwords
rem_stopwords_tokenize(data,'Review_without_stopwords')
# Converting all the texts back to sentences
make_sentences(data,'Review_without_stopwords')


#Edits After Lemmatization
final_Edit = data['Review_without_stopwords'].copy()
data["After_lemmatization"] = final_Edit
Lemmatization(data,'After_lemmatization')

# Converting all the texts back to sentences
make_sentences(data,'After_lemmatization')

# Results of Preprocessing data (Removing stopwords & Lemmatization)

In [ ]:
data.head(6)

In [ ]:
print("- Old Review -")
print(data['Review'][3])
print("\n- New Review -")
print(data['Review_without_stopwords'][3])
print("\n- Last Edit Review -")
print(data['After_lemmatization'][3])

In [ ]:
data['reviews_text_new'] = data['After_lemmatization'].copy()

In [ ]:
# Mapping the ratings
data['Sentiment_rating'] = np.where(data.Rating >= 3,1,0)

# Printing the counts of each class
data['Sentiment_rating'].value_counts()

In [ ]:
data.head(5)

In [ ]:
# Replacing Positive -> 1 and Negative -> 0

data.replace({"positive":1,"negative":0},inplace=True)

In [ ]:
data[['reviews_text_new','Sentiment_rating']].head(5)

# Building a machine learning model

# Bag-of-words and n-grams

# Divide into training and test sets:

# Applying logistic regression

In [ ]:
bow_counts = CountVectorizer(tokenizer= word_tokenize,
                             lowercase=True,
                             ngram_range=(1,1))

bow_data = bow_counts.fit_transform(data.reviews_text_new)

In [ ]:
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(bow_data,
                                                                    data['Sentiment_rating'],
                                                                    test_size = 0.2,
                                                                    random_state = 0,
                                                                    shuffle=False,
                                                                   stratify=None)

In [ ]:
# Defining and training the model
lr_model_all_new = LogisticRegression(max_iter = 200)
lr_model_all_new.fit(X_train_bow, y_train_bow)

# Predicting the results
test_pred_lr_all = lr_model_all_new.predict(X_test_bow)


## Calculate key performance metrics

# Print a classification report
print(classification_report(y_test_bow,test_pred_lr_all))

In [ ]:
X_train_senti, X_test_senti, y_train_senti, y_test_senti = train_test_split(data['reviews_text_new'],
                                                                            data['Sentiment_rating'],
                                                                            test_size = 0.2,
                                                                            random_state = 0,
                                                                           shuffle=False,
                                                                           stratify=None)

In [ ]:
# Create A New DataFrame For Testing And Analysing 

df_test = pd.DataFrame(columns = ['review_test','actual_score', 'lr_score','swn_score'])
df_test['review_test'] = X_test_senti
df_test['actual_score'] = y_test_senti
df_test['lr_score'] = test_pred_lr_all
df_test.head(5)

In [ ]:
import nltk
import ssl
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [ ]:
pos=neg=obj=count=0

postagging = []

for review in X_test_senti:
    list = word_tokenize(review)
    postagging.append(nltk.pos_tag(list))

df_test['pos_tags'] = postagging

def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None


# Returns list of pos-neg and objective score. But returns empty list if not present in senti wordnet.
def get_sentiment(word,tag):
    wn_tag = penn_to_wn(tag)
    
    if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
        return []

    #Lemmatization
    lemma = lemmatizer.lemmatize(word, pos=wn_tag)
    if not lemma:
        return []

    #Synset is a special kind of a simple interface that is present in NLTK to look up words in WordNet. 
    #Synset instances are the groupings of synonymous words that express the same concept. 
    #Some of the words have only one Synset and some have several.
    synsets = wn.synsets(word, pos=wn_tag)
    if not synsets:
        return []

    # Take the first sense, the most common
    synset = synsets[0]
    swn_synset = swn.senti_synset(synset.name())

    return [synset.name(), swn_synset.pos_score(),swn_synset.neg_score(),swn_synset.obj_score()]

    pos=neg=obj=count=0
    
    ###################################################################################
senti_score = []

for pos_val in df_test['pos_tags']:
    senti_val = [get_sentiment(x,y) for (x,y) in pos_val]
    for score in senti_val:
        try:
            pos = pos + score[1]  #positive score is stored at 2nd position
            neg = neg + score[2]  #negative score is stored at 3rd position
        except:
            continue
    senti_score.append(pos - neg)
    pos=neg=0    
    
df_test['senti_score'] = senti_score

In [ ]:
len(df_test)

In [ ]:
df_test.head(5)

In [ ]:
overall=[]
for i in range(16392,20491,1):
    if df_test['senti_score'][i]>= 0:
        overall.append(1)
    elif df_test['senti_score'][i]< 0:
        overall.append(0)
    
df_test['swn_score']=overall

In [ ]:
df_test.head(10)

In [ ]:
case1=case2=case3=case4=case5=case6=0
for i in range(16392,20491,1):
    if ((df_test['lr_score'][i] == df_test['swn_score'][i]) and (df_test['swn_score'][i] == df_test['actual_score'][i])):
        case1 = case1+1
    if ((df_test['lr_score'][i] == df_test['swn_score'][i]) and (df_test['swn_score'][i] != df_test['actual_score'][i])):
        case2 = case2+1
    if ((df_test['lr_score'][i] != df_test['swn_score'][i]) and (df_test['lr_score'][i] == df_test['actual_score'][i])):
        case3 = case3+1
    if ((df_test['lr_score'][i] != df_test['swn_score'][i]) and (df_test['swn_score'][i] == df_test['actual_score'][i])):
        case4 = case4+1
    if ((df_test['lr_score'][i] != df_test['swn_score'][i]) and (df_test['actual_score'][i]== 0)):
        case5 = case5+1
    if ((df_test['lr_score'][i] != df_test['swn_score'][i]) and (df_test['actual_score'][i]==1)):
        case6 = case6+1
        
print("case 1",case1) # 58%
print("case 2",case2) # 5%
print("case 3",case3) # 30%
print("case 4",case4) # 5%
print("case 5",case5) # 25%
print("case 6",case6) # 10%


In [ ]:
from sklearn.metrics import accuracy_score
lr = accuracy_score(y_test_bow, test_pred_lr_all)
swn = accuracy_score(y_test_bow, overall)
print("lr_accuracy",lr)
print("swn_accuracy",swn)

In [ ]:
final_score =[]
for i in range(16392,20491,1):
    if 0.58*(df_test['lr_score'][i] == df_test['swn_score'][i]) or 0.3*(df_test['lr_score'][i] != df_test['swn_score'][i]):
        final_score.append(df_test['lr_score'][i])
    elif 0.05*(df_test['lr_score'][i] != df_test['swn_score'][i]):
        final_score.append(df_test['swn_score'][i])
    elif 0.25*(df_test['lr_score'][i] != df_test['swn_score'][i]):
        final_score.append(0)
    elif 0.1*(df_test['lr_score'][i] != df_test['swn_score'][i]):
        final_score.append(1)
        
df_test['final_score_upt']=final_score

In [ ]:
final_upt = accuracy_score(y_test_bow, final_score)
print("final_hyprid_accuracy",final_upt)

In [ ]:
final_score1 =[]
for i in range(16392,20491,1):
    if (df_test['lr_score'][i]==1) and (df_test['swn_score'][i]==1):
        final_score1.append(1)
    elif (df_test['lr_score'][i]==0) and (df_test['swn_score'][i]==0):
        final_score1.append(0)
    else :
        final_score1.append(df_test['lr_score'][i])
df_test['final_score_upt']=final_score1

In [ ]:
final_upt1 = accuracy_score(y_test_bow, final_score1)
print("final_hyprid_accuracy",final_upt1)

In [ ]:
df_train = pd.DataFrame(columns = ['lr_score','swn_score'])
df_train['swn_score'] = overall
df_train['lr_score'] = test_pred_lr_all
df_train.head(4)

In [ ]:
X_trainf, X_testf, y_trainf, y_testf = train_test_split(df_train,
                                                                    df_test['actual_score'],
                                                                    test_size = 0.2,
                                                                    random_state = 0,
                                                                    shuffle=False,
                                                                   stratify=None)



In [ ]:
hyprid = LogisticRegression(max_iter = 200).fit(X_trainf,y_trainf).predict(X_testf)

In [ ]:
acc = accuracy_score(y_testf, hyprid)
print("accuracy",acc)

In [ ]:
#lr_model_all_new.feature_names=bow_counts.get_feature_names()

In [ ]:
#from joblib import dump, load 

# save model to file 
#dump(lr_model_all_new, filename="Sentiment_Analysis_unigram2.joblib")

In [ ]:
# import a saved joblib model 
#loaded_joblib_model = load(filename="Sentiment_Analysis_unigram2.joblib")

In [ ]:
#len(loaded_joblib_model.feature_names)

In [ ]:
#import string
#from nltk import word_tokenize
#from nltk.corpus import stopwords
#from nltk.stem import WordNetLemmatizer
#from nltk.stem import WordNetLemmatizer
#feats = bow_counts.get_feature_names()
#feats_len = len(feats)
#sent ='You have to build a very bad than doing any thing else#'
#sent =sent.lower()
#sent = sent.translate(str.maketrans('', '', string.punctuation))
#filtered_sentence = [] 
#stop_words = set(stopwords.words('english')) 
#word_tokens =word_tokenize(sent)
#filtered_sentence = [w for w in word_tokens if not w in stop_words ]
#listToStr = ' '.join(map(str, filtered_sentence))
#lemmatizer = WordNetLemmatizer()
#wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
#def lemmatize_words(text):
    #pos_tagged_text = nltk.pos_tag(word_tokenize(text))
    #return ([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])
#lemmatized_output =[]
#lemmatized_output = lemmatize_words(listToStr)

#sent_features=[]
#sent_dict = {}
#for word in lemmatized_output:
    #if not word in sent_dict:
        #sent_dict[word] = 0
    #sent_dict[word] = sent_dict[word] + 1
#for i in range(feats_len):
    #if not feats[i] in sent_dict:
        #sent_features.append(0)
    #else:
        #sent_features.append(sent_dict[feats[i]])

In [ ]:
#len(feats)

In [ ]:
#len(sent_features)

In [ ]:
#joblib_y_preds = loaded_joblib_model.predict([sent_features])

In [ ]:
#print(joblib_y_preds)